In [ ]:
!pip3 install numpy --upgrade
!pip3 install scipy --upgrade

In [1]:
import fft_project
import psutil
import logging
import time
import scipy
import numpy as np
from importlib import reload
from fft_project import base


In [2]:
'''Below are the testcases, where we controlled the multithreading and creating copies
of our version of 2DFFT versus Scipy fft2'''

#Printing out Current CPU Info
print("="*40, "CPU Info", "="*40)
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")

======================================== CPU Info ========================================
Physical cores: 10
Total cores: 10
Max Frequency: 3228.00Mhz
Min Frequency: 600.00Mhz
Current Frequency: 3228.00Mhz


In [3]:
def random_matrix(n, m):
    logging.info(f'Matrix shape: {n}x{m}')
    result = np.zeros((n, m), dtype=np.complex128)
    result.real = np.random.rand(n, m)
    return result

reload(logging)
base.prepare_logger()


Measurements with multithreading disabled, and no type conversions

In [4]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    start = time.time()
    expected = scipy.fft.fft2(result)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    start = time.time()
    result = fft_project.fft2d(result, return_copy=False, use_threads=False)
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    expected = abs(expected)
    result = abs(result)
    correctness = np.allclose(result, expected, atol=0.05)
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=False, use_threads=False)
    start = time.time()
    inversed_scipy = scipy.fft.ifft2(transformed, workers = -1)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    inversed_our = transformed
    start = time.time()
    inversed_our = fft_project.fft2d(
        inversed_our, return_copy=False, use_threads=False, inverse=True
    )
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    inversed_scipy = abs(inversed_scipy)
    inversed_our = abs(inversed_our)
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    assert correctness

fft_cpp_impl_test()
inverse_test()

[INFO]  Matrix shape: 4096x8192
[INFO]  Scipy: 0.7608420848846436
[INFO]  Our: 0.3916947841644287
[INFO]  51.48% of Scipy time!
[INFO]  Matrix shape: 8192x4096
[INFO]  Scipy: 0.12591218948364258
[INFO]  Our: 0.37282299995422363
[INFO]  296.10% of Scipy time!


With multithreading enabled, no type conversions

In [5]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    start = time.time()
    expected = scipy.fft.fft2(result)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    start = time.time()
    result = fft_project.fft2d(result, return_copy=False, use_threads=True)
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    expected = abs(expected)
    result = abs(result)
    correctness = np.allclose(result, expected, atol=0.05)
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=False, use_threads=True)
    start = time.time()
    inversed_scipy = scipy.fft.ifft2(transformed, workers = -1)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    inversed_our = transformed
    start = time.time()
    inversed_our = fft_project.fft2d(
        inversed_our, return_copy=False, use_threads=True, inverse=True
    )
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    inversed_scipy = abs(inversed_scipy)
    inversed_our = abs(inversed_our)
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    assert correctness

fft_cpp_impl_test()
inverse_test()

[INFO]  Matrix shape: 4096x8192
[INFO]  Scipy: 0.5515410900115967
[INFO]  Our: 0.14614367485046387
[INFO]  26.50% of Scipy time!
[INFO]  Matrix shape: 8192x4096
[INFO]  Scipy: 0.13767194747924805
[INFO]  Our: 0.14137005805969238
[INFO]  102.69% of Scipy time!


No multithreading, with type conversions

In [6]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    start = time.time()
    expected = scipy.fft.fft2(result)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    start = time.time()
    result = fft_project.fft2d(result, return_copy=True, use_threads=False)
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    expected = abs(expected)
    result = abs(result)
    correctness = np.allclose(result, expected, atol=0.05)
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=True, use_threads=False)
    start = time.time()
    inversed_scipy = scipy.fft.ifft2(transformed, workers = -1)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    inversed_our = transformed
    start = time.time()
    inversed_our = fft_project.fft2d(
        inversed_our, return_copy=True, use_threads=False, inverse=True
    )
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    inversed_scipy = abs(inversed_scipy)
    inversed_our = abs(inversed_our)
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    assert correctness

fft_cpp_impl_test()
inverse_test()

[INFO]  Matrix shape: 4096x8192
[INFO]  Scipy: 0.5422320365905762
[INFO]  Our: 0.462644100189209
[INFO]  85.32% of Scipy time!
[INFO]  Matrix shape: 8192x4096
[INFO]  Scipy: 0.12778401374816895
[INFO]  Our: 0.464385986328125
[INFO]  363.41% of Scipy time!


No multithreading, no type conversions

In [7]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    start = time.time()
    expected = scipy.fft.fft2(result)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    start = time.time()
    result = fft_project.fft2d(result, return_copy=False, use_threads=False)
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    expected = abs(expected)
    result = abs(result)
    correctness = np.allclose(result, expected, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=False, use_threads=False)
    start = time.time()
    inversed_scipy = scipy.fft.ifft2(transformed, workers = -1)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    inversed_our = transformed
    start = time.time()
    inversed_our = fft_project.fft2d(
        inversed_our, return_copy=True, use_threads=False, inverse=True
    )
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    inversed_scipy = abs(inversed_scipy)
    inversed_our = abs(inversed_our)
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

fft_cpp_impl_test()
inverse_test()


[INFO]  Matrix shape: 4096x8192
[INFO]  Scipy: 0.5387053489685059
[INFO]  Our: 0.3598310947418213
[INFO]  66.80% of Scipy time!
[INFO]  Is the result correct: True
[INFO]  Matrix shape: 8192x4096
[INFO]  Scipy: 0.13294386863708496
[INFO]  Our: 0.4173769950866699
[INFO]  313.95% of Scipy time!
[INFO]  Is the result correct: True
